In [19]:
from transformers import BertTokenizer
import logging
from datetime import datetime
import uuid
from sklearn.model_selection import StratifiedKFold
import tensorflow.compat.v1 as tf

In [20]:
from Resources.DCNN import DCNN
from Resources.Utils import Utils

In [21]:
# import numpy as np
# import pandas as pd
# import random
# import matplotlib.pyplot as plt
# from tensorflow.keras import layers
# import tensorflow.compat.v1 as tf
# import wandb
# import os

# from sklearn import preprocessing

# from wandb.keras import WandbCallback
# from Resources.Models import DCNN
# from Resources.Utils import Utils
# 
# import uuid
# from sklearn import preprocessing
# import wandb
# from dotenv import load_dotenv
# import multiprocessing
# import collections
# from datetime import datetime

In [25]:
language = 'asl'
# language = 'germany'
# language = 'libras'

In [23]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path = "Outputs/tokenizers/DistilBertTokenizerFastSW")
logging.basicConfig(level=logging.INFO)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
date = datetime.today().strftime('%Y-%m-%d')

sweep_id =  '{}-{}'.format(date,str(uuid.uuid1()))
u = Utils()
clean_tweet = u.clean_tweet
encode_sentence = u.encode_sentence
iteracoesDataset = u.iteracoesDataset

c = {'EMB_DIM': 512,
    'PADDING': 'same',
    'dataset': 'details',
    'FFN_UNITS': 512,
    'NB_EPOCHS': 200,
    'size_desc': 4,
    'NB_FILTERS': 150,
    'batch_size': 32,
    'repet_test': 6,
    'repet_train': 10-1,
    'DROPOUT_RATE': 0.1}
k = 10


sweep_id =  '{}{}-{}-{}'.format(language,k,date,str(uuid.uuid1()).split('-')[0])
# Carrega o dataset
data = u.load_db(language, c['dataset'])
# seleciona a subamostragem desejada
data = u.selectSubGroupDatabase(data, c['repet_train'])

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
skf.get_n_splits(data.sentence, data.label)

K_test_dataset = []
K_train_dataset = []

for train_index, test_index in skf.split(data.sentence, data.label):
    K_train_dataset.append(data.iloc[train_index])
    K_test_dataset.append(data.iloc[test_index])
    num_labels = len(data.label.unique())

for i in range(len(K_train_dataset)):
    run_name = "{}-{}".format(language, i)
    # run = wandb.init(project="kfold-bert", group=sweep_id, name=run_name, job_type = language, config=c)

    callbacks = []
    callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3))

    # callbacks.append(WandbCallback())

    train_dataset = K_train_dataset[i]
    test_dataset = K_test_dataset[i]

    train_dataset, test_dataset = iteracoesDataset(train_dataset, test_dataset, data, tokenizer,c)
    VOCAB_SIZE = len(tokenizer.vocab)

    NB_CLASSES = num_labels

    Dcnn = DCNN(vocab_size=VOCAB_SIZE,
                emb_dim=c['EMB_DIM'],
                nb_filters=c['NB_FILTERS'],
                FFN_units=c['FFN_UNITS'],
                nb_classes=NB_CLASSES,
                dropout_rate=c['DROPOUT_RATE'],
                padding=c['PADDING'])

    Dcnn.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam', metrics=['sparse_categorical_accuracy'])

    checkpoint_path = 'temp'
    ckpt = tf.train.Checkpoint(Dcnn=Dcnn)
    ckpt_manager = tf.train.CheckpointManager(
        ckpt, checkpoint_path, max_to_keep=1)

    history = Dcnn.fit(train_dataset,
                    epochs=c['NB_EPOCHS'],
                    callbacks=callbacks)

    results = Dcnn.evaluate(test_dataset)

    top_pred_ids = Dcnn.predict(test_dataset)

    r_predict = []
    for item in top_pred_ids:
        r_predict.append(u.get_prediction(item))

    ground_truth_ids = []
    for element in test_dataset.as_numpy_iterator():
        aux = element[1]
        for item in aux:
            ground_truth_ids.append(item)

    # try:
    #     # run.log({"{}".format(run_name) : wandb.plot.confusion_matrix(
    #                 # preds=r_predict, y_true=ground_truth_ids,
    #                 # class_names=le.inverse_transform(ground_truth_ids))})
    # except:
    #     print("Erro no passo {}".format(run_name))

    print({'test_loss': results[0], 'test_sparse_categorical_accuracy': results[1]})
    # run.finish()

2023-07-09 07:06:56.049782: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was int32, but the yielded element was [0, None, None, None, None, 14, None, 14, None, 14, None, None, None, 14].
Traceback (most recent call last):

  File "/home/fernando/anaconda3/envs/bracis/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 214, in generator_py_func
    script_ops.FuncRegistry._convert(  # pylint: disable=protected-access

  File "/home/fernando/anaconda3/envs/bracis/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 227, in _convert
    result = np.asarray(value, dtype=dtype, order="C")
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


The above exception was the direct cause of the following exception:


Tr

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was int32, but the yielded element was [0, None, None, None, None, 14, None, 14, None, 14, None, None, None, 14].
Traceback (most recent call last):

  File "/home/fernando/anaconda3/envs/bracis/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 214, in generator_py_func
    script_ops.FuncRegistry._convert(  # pylint: disable=protected-access

  File "/home/fernando/anaconda3/envs/bracis/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 227, in _convert
    result = np.asarray(value, dtype=dtype, order="C")
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


The above exception was the dire